<a href="https://colab.research.google.com/github/AndreGulyi/Computer-Vision-Matarials/blob/main/Auto_Data_Annotation_for_image_segmentation_AGtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://segment-anything.com/

In [2]:
!nvidia-smi

Fri Aug 15 13:11:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!pip install ultralytics==8.0.106

In [7]:
import ultralytics

 - sam_l.pt
 - sam_b.pt

In [11]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [12]:
from ultralytics.vit import SAM

model = SAM('sam_b.pt')

model.predict('https://drive.google.com/file/d/12F8wMunQqDi-GhEtvwVKha76DUA5__pq/view?usp=drive_link')

Ultralytics YOLOv8.0.106 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)



ConnectionError: 1/1: https://drive.google.com/file/d/12F8wMunQqDi-GhEtvwVKha76DUA5__pq/view?usp=drive_link... Failed to open https://drive.google.com/file/d/12F8wMunQqDi-GhEtvwVKha76DUA5__pq/view?usp=drive_link

In [ ]:
from ultralytics.vit import SAM
model = SAM('sam_b.pt')
results = model.predict('video/kids.mp4', device = 0)

In [ ]:
# webcam
from ultralytics.vit import SAM
model = SAM("sam_l.pt")
results = model.predict(0, device = 0, show = True)

# Generate Segmentation Dataset Using a Detection Model

https://github.com/ultralytics/ultralytics

In [ ]:
from ultralytics.yolo.data.annotator import auto_annotate

auto_annotate(data="images", det_model="yolov8x.pt", sam_model='sam_b.pt')


# Code for auto annotation

In [ ]:
import cv2
import numpy as np
from pathlib import Path

from ultralytics import YOLO
from ultralytics.vit.sam import PromptPredictor, build_sam
from ultralytics.yolo.utils.torch_utils import select_device

import matplotlib.pyplot as plt

def auto_annotate(data, det_model='yolov8x.pt', sam_model='sam_b.pt', device='', output_dir=None):
    """
    Automatically annotates images using a YOLO object detection model and a SAM segmentation model.
    Args:
        data (str): Path to a folder containing images to be annotated.
        det_model (str, optional): Pre-trained YOLO detection model. Defaults to 'yolov8x.pt'.
        sam_model (str, optional): Pre-trained SAM segmentation model. Defaults to 'sam_b.pt'.
        device (str, optional): Device to run the models on. Defaults to an empty string (CPU or GPU, if available).
        output_dir (str, None, optional): Directory to save the annotated results.
            Defaults to a 'labels' folder in the same directory as 'data'.
    """
    device = select_device(device)
    det_model = YOLO(det_model)
    sam_model = build_sam(sam_model)
    det_model.to(device)
    sam_model.to(device)

    if not output_dir:
        output_dir = Path(str(data)).parent / 'labels'
    Path(output_dir).mkdir(exist_ok=True, parents=True)

    prompt_predictor = PromptPredictor(sam_model)
    det_results = det_model(data, stream=True)

    for result in det_results:
        boxes = result.boxes.xyxy  # Boxes object for bbox outputs
        class_ids = result.boxes.cls.int().tolist()  # noqa
        if len(class_ids):
            prompt_predictor.set_image(result.orig_img)
            masks, _, _ = prompt_predictor.predict_torch(
                point_coords=None,
                point_labels=None,
                boxes=prompt_predictor.transform.apply_boxes_torch(boxes, result.orig_shape[:2]),
                multimask_output=False,
            )



            result.update(masks=masks.squeeze(1))
            segments = result.masks.xyn  # noqa



            with open(str(Path(output_dir) / Path(result.path).stem) + '.txt', 'w') as f:
                for i in range(len(segments)):
                    s = segments[i]
                    if len(s) == 0:
                        continue
                    segment = map(str, segments[i].reshape(-1).tolist())
                    f.write(f'{class_ids[i]} ' + ' '.join(segment) + '\n')
